# 第4回課題
内包表記，並列処理

In [ ]:
#!pip install joblib

In [2]:
from joblib import Parallel, delayed
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

## 内包表記
python における高速化テクの1つである内包表記の実装（for 文の削減）

### 課題3

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    for j in range(4):
        data.append(i*j)
data
```
出力：
[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

In [3]:
data = [i*j for i in range(5) for j in range(4)]
data

[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

### 課題2

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    inner = []
    for j in range(i, 6):
        inner.append(i)
    data.append(inner)
data
```
出力：
[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

In [4]:
data = [[i for j in range(i, 6)] for i in range(5)]
data

[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

## 並列処理
python における高速化テクの1つである並列化の実装（複数コア，CPU の使用）

joblib だけでなく multiprocessing もある

### 例：2乗の場合

In [5]:
%%timeit 3 # 3回下記のコードを実行した時の時間を表示
r = []
for i in range(10000):
    r.append(i ** 2)
r[:3] + r[-3:]

2.2 ms ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
?Parallel

Init signature:
Parallel(
    n_jobs=None,
    backend=None,
    verbose=0,
    timeout=None,
    pre_dispatch='2 * n_jobs',
    batch_size='auto',
    temp_folder=None,
    max_nbytes='1M',
    mmap_mode='r',
    prefer=None,
    require=None,
)
Docstring:     
Helper class for readable parallel mapping.

Read more in the :ref:`User Guide <parallel>`.

Parameters
-----------
n_jobs: int, default: None
    The maximum number of concurrently running jobs, such as the number
    of Python worker processes when backend="multiprocessing"
    or the size of the thread-pool when backend="threading".
    If -1 all CPUs are used. If 1 is given, no parallel computing code
    is used at all, which is useful for debugging. For n_jobs below -1,
    (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all
    CPUs but one are used.
    None is a marker for 'unset' that will be interpreted as n_jobs=1
    (sequential execution) unless the call is performed under a
    parallel_backend context mana

In [7]:
%%timeit 3 
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=0)([delayed(power)(i) for i in range(10000)])
r[:3] + r[-3:]

280 ms ± 7.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=3)( [delayed(power)(i) for i in range(10000)] )
r[:3] + r[-3:]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 8160 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:    0.2s finished


[0, 1, 4, 99940009, 99960004, 99980001]

In [9]:
%%timeit 3
# numpy が最速というのはよくある
np.arange(10000) ** 2

10.7 µs ± 90.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%%timeit 3
np.fromiter(range(10000), dtype=np.int) ** 2

214 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 課題1

x = [(i, i+1) for i in range(100000)] としたとき，各 index 毎に積を求めよ

出力 [0, 2, 6,（中略） 9999500006, 9999700002, 9999900000]

In [11]:
mul = lambda x, y: x * y
r = Parallel(n_jobs=-1, verbose=0)(delayed(mul)(i, i+1) for i in range(100000))
r[:3] + r[-3:]

[0, 2, 6, 9999500006, 9999700002, 9999900000]

### 発展課題
C = np.cumsum(np.arange(1000)) の各項の差を求めよ

出力[1, 2, 3,（中略） 997, 998, 999] = np.arange(1, 1000)

In [21]:
C = np.cumsum(np.arange(1000))
sub = lambda x, y: y - x
r = Parallel(n_jobs=-1, verbose=0)(delayed(sub)(C[i-1], C[i]) for i in range(1, 1000))
r[:3] + r[-3:]

[1, 2, 3, 997, 998, 999]